In [1]:
import sys
sys.path.append("../..")

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
# environments

from environment.qlearning.obstacle_avoidance_env import ObstacleAvoidanceEnv

In [4]:
from utils.reader import get_yaml_path, read_file

In [5]:
from agent.qagent import QAgent

In [6]:
from evaluation.agent_evaluation import evaluate

2025-11-28 11:40:13.142832: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-28 11:40:13.176593: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-28 11:40:16.374195: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [7]:
from utils.plotting import plot_all_q_agent

In [8]:
server_address = "localhost:50051" # adjust the port as needed
client_name = "RLClient"
env = ObstacleAvoidanceEnv(server_address, client_name) # adjust the env as needed
env.connect_to_client()

2025-11-28 11:40:17,320 — INFO — ✓ Connected to localhost:50051



In [9]:
agents = {
    f"00000000-0000-0000-0000-{(i+1):012d}": QAgent(env)
    for i in range(30)
}
# agent = QAgent(env)
# agent_id = "00000000-0000-0000-0000-000000000001"
# agents = { agent_id: agent }

In [10]:
configs_path = get_yaml_path("src", "scripts", "resources", "generated", "evaluation", "q-learning", "obstacle-avoidance", "multi-agent")
yml_files = sorted(configs_path.glob("*.yml"), key=lambda p: p.name)
configs = [read_file(f) for f in yml_files]
print(len(configs))

10


In [11]:
agent_path = get_yaml_path("src", "scripts", "resources", "generated", "checkpoints", "obstacle-avoidance", "q-learning", "v1", "oa_ep884")
for agent in agents.values():
    agent.load(agent_path)

2025-11-28 11:40:17,336 — INFO — Agent loaded from /home/simone/uni-lab/PPS/2024/SRS/PPS-22-srs/docs/obstacle-avoidance/python/src/scripts/resources/generated/checkpoints/obstacle-avoidance/q-learning/v1/oa_ep884.npz
2025-11-28 11:40:17,337 — INFO —   Q-table shape: (64, 5)
2025-11-28 11:40:17,337 — INFO —   Current epsilon: 0.1318
2025-11-28 11:40:17,337 — INFO —   Total episodes trained: 884
2025-11-28 11:40:17,338 — INFO — Agent loaded from /home/simone/uni-lab/PPS/2024/SRS/PPS-22-srs/docs/obstacle-avoidance/python/src/scripts/resources/generated/checkpoints/obstacle-avoidance/q-learning/v1/oa_ep884.npz
2025-11-28 11:40:17,338 — INFO —   Q-table shape: (64, 5)
2025-11-28 11:40:17,338 — INFO —   Current epsilon: 0.1318
2025-11-28 11:40:17,339 — INFO —   Total episodes trained: 884
2025-11-28 11:40:17,339 — INFO — Agent loaded from /home/simone/uni-lab/PPS/2024/SRS/PPS-22-srs/docs/obstacle-avoidance/python/src/scripts/resources/generated/checkpoints/obstacle-avoidance/q-learning/v1/oa

In [12]:
did_succeed=lambda reward, termination, truncation: True if truncation and reward > -500 else False

In [ ]:
results = evaluate(env=env, agents=agents, configs=configs, max_steps=5000, did_succeed=did_succeed) # TODO: 5000 steps

Evaluation:  30%|█████████████████████████████████████████████████████                                                                                                                            | 3/10 [01:21<03:10, 27.16s/configuration run]

In [ ]:
plot_all_q_agent(results, agents=list(agents.keys()))